<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Llama_3_8B_for_text_to_SQL_upload_model_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

In [ ]:
!pip install colab-env --quiet
!pip install huggingface_hub -q

import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

Mounted at /content/gdrive


In [ ]:
#!pip install huggingface_hub --quiet
from huggingface_hub import HfApi

api = HfApi()
api.get_token_permission(token=access_token_write)
#api.set_access_token(access_token)


# frankmorales2020/Mistral-7B-text-to-sql Good

repo_id = "my-awesome-model-poc"
#api.create_repo(repo_id=repo_id, private=False)

#api.delete_repo(repo_id=repo_id)

#api.upload_folder(
#    folder_path="./model",
#    repo_id=repo_id,
#    repo_type="model",
#)


In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)

In [ ]:
#frankmorales2020/Mistral-7B-text-to-sql
# Chose the base model you want
#model_name = "frankmorales2020/Mistral-7B-text-to-sql" # 01/03/2023
#frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2
model_name = "frankmorales2020/Meta-Llama-3-8B-text-to-sql-flash-attention-2" # 04/03/2024
# set device
device = 'cuda'
#v Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install huggingface_hub -q

In [ ]:
from datasets import load_dataset
from huggingface_hub import hf_hub_url

In [ ]:
from datasets import load_dataset

# Convert dataset to OAI messages
system_message = """You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}"""

def create_conversation(sample):
  return {
    "messages": [
      {"role": "system", "content": system_message.format(schema=sample["context"])},
      {"role": "user", "content": sample["question"]},
      {"role": "assistant", "content": sample["answer"]}
    ]
  }

# Load dataset from the hub
# jtjt520j/CSpider_sql_create_context
# b-mc2/sql-create-context
#dataset = load_dataset("jtjt520j/CSpider_sql_create_context", split="train")
dataset = load_dataset("b-mc2/sql-create-context", split="train")
dataset = dataset.shuffle().select(range(12500))

# Convert dataset to OAI messages
dataset = dataset.map(create_conversation, remove_columns=dataset.features,batched=False)
# split dataset into 10,000 training samples and 2,500 test samples
dataset = dataset.train_test_split(test_size=2500/12500)

print(dataset["train"][345]["messages"])

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

In [ ]:
from datasets import load_dataset
from random import randint

# Load our test dataset and Tokenizer again
#tokenizer = AutoTokenizer.from_pretrained("frankmorales2020/Mistral-7B-text-to-sql") # 01/03/2024
tokenizer = AutoTokenizer.from_pretrained("frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2") # 04/03/2024
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

In [ ]:
# generate the same prompt as for the first local test
prompt = tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
request= {"inputs":prompt,"parameters":{"temperature":0.2, "top_p": 0.95, "max_new_tokens": 256}}


Parameter-Efficient Fine-Tuning (PEFT)

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

#peft_model_id = "frankmorales2020/Mistral-7B-text-to-sql" # 01/03/2024
peft_model_id = "frankmorales2020/Mistral-7B-text-to-sql-flash-attention-2" # 04/03/2024 and 10/03/2024
# peft_model_id = args.output_dir

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
from datasets import load_dataset
from random import randint


# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

# Test on sample
prompt = pipe.tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.1, top_k=50, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)

In [ ]:
print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
Name the transit passengers for 171078
Original Answer:
SELECT MAX(transit_passengers) FROM table_13836704_8 WHERE freight__metric_tonnes_ = 171078
Generated Answer:
SELECT MAX(transit_passengers) FROM table_13836704_8 WHERE freight__metric_tonnes_ = 171078


https://huggingface.co/frankmorales2020

In [ ]:
from tqdm import tqdm

# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()
    print()
    print()
    print('Question: %s\n'%sample["messages"][1]["content"])
    #print(sample["messages"][2]["content"])
    #print('Predicted Answer: %s'%sample["messages"][2]["content"])
    print()
    if predicted_answer == sample["messages"][2]["content"]:
        #print('Success!')
        print('Success Answer: %s'%sample["messages"][2]["content"])
        return 1
    else:
        print('Real Answer: %s'%predicted_answer)
        print('Failed Answer: %s'%sample["messages"][2]["content"])
        return 0

success_rate = []

number_of_eval_samples = 10
# iterate over eval dataset and predict
for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

print(f"Accuracy: {accuracy*100:.2f}%")

 10%|█         | 1/10 [00:01<00:17,  1.94s/it]



Question: What yacht did Andrew Saies sail on?


Success Answer: SELECT yacht FROM table_25561560_3 WHERE skipper = "Andrew Saies"


 20%|██        | 2/10 [00:05<00:25,  3.16s/it]



Question: What is the record at the opponent's venue against the team for which the record for the last 5 meetings is mu, 4-1 and the record at the neutral site is tied, 0-0?


Success Answer: SELECT at_opponents_venue FROM table_16201038_4 WHERE last_5_meetings = "MU, 4-1" AND at_neutral_site = "Tied, 0-0"


 30%|███       | 3/10 [00:07<00:15,  2.24s/it]

...



Question: Count the Number of electorates (2009) that has a District of fatehpur and  jahanabad?


Real Answer: SELECT MIN(number_of_electorates__2009_) FROM table_name_11 WHERE district = "fatehpur" AND name = "jahanabad"
Failed Answer: SELECT SUM(number_of_electorates__2009_) FROM table_name_11 WHERE district = "fatehpur" AND name = "jahanabad"


 90%|█████████ | 9/10 [00:21<00:02,  2.46s/it]



Question: What transfer fee has both winter as the transfer window, and Madureira as the moving to?


Success Answer: SELECT transfer_fee FROM table_name_68 WHERE transfer_window = "winter" AND moving_to = "madureira"


100%|██████████| 10/10 [00:23<00:00,  2.36s/it]



Question: Which Xenon has Neon of 20.1797(6)?


Success Answer: SELECT xenon FROM table_name_25 WHERE neon = "20.1797(6)"
Accuracy: 90.00%


When evaluated on 10 samples from the evaluation dataset, our model achieved an impressive accuracy of 90.00%.


In [ ]:
from tqdm import tqdm

# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()
    if predicted_answer == sample["messages"][2]["content"]:
        return 1
    else:
        return 0

success_rate = []
number_of_eval_samples = 1000
# iterate over eval dataset and predict
for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

print(f"Accuracy: {accuracy*100:.2f}%")

100%|██████████| 1000/1000 [33:17<00:00,  2.00s/it]

Accuracy: 82.60%



When evaluated on 1000 samples from the evaluation dataset, our model achieved an impressive accuracy of 82.60%. However, there's room for improvement. We could enhance the model's performance by exploring techniques like few-shot learning, RAG, and Self-healing to generate the SQL query.